In [1]:
from sympy import symbols, poly, Sum, Indexed, sympify, Poly, Lambda, Derivative, Function, Subs, lambdify, DiracDelta, series
from sympy.functions import sign, Abs
import numpy as np
from scipy.special import gamma, factorial
import pandas as pd

In [2]:
x, y, z, p, s, t = symbols("x y z p s t", real = True)

In [3]:
M = 6
v = symbols(f"v:{M}", real = True)
v_sub = symbols(f"v_sub:{M}", real = True)
d, g = symbols("delta gamma", real = True)
f = Lambda(x, x ** d)
f_dev = Lambda(x, d * x ** (d - 1))
poly_p = Poly(v[::-1], p)
poly_p_sub = Poly(v_sub[::-1], p)
v_func = poly_p.expr
v_sub_func = poly_p_sub.expr
F_first = Lambda(p, f(v_func))
F_second = Lambda(p, v_func * f_dev(v_sub_func))

In [5]:
h = -50.0
N = 50
gamma_const = 0.5
delta_const = 0.5
T = 1
X = 0.1
G = np.zeros((N, N))

def G_ij_upper(i, j):
    return (1 / (1 - gamma_const) * (1 - gamma_const)) * (T / N) ** (2 - gamma_const) \
            * ((i - j + 1) ** (2 - gamma_const) - 2 * (i - j) ** (2 - gamma_const) +\
                 (i - j - 1) ** (2 - gamma_const)) 

def G_ii_diagonal():
    return (2 / ((1 - gamma_const) * (2 - gamma_const))) * (T / N) ** (2 - gamma_const)
    
G_upper = np.fromfunction(G_ij_upper, (N, N))
G_upper = np.nan_to_num(G_upper, nan = 0.0)
G_diag = np.repeat(G_ii_diagonal(), N)
G_diag = np.diag(G_diag)
G = G_upper.T + G_upper + G_diag

v_guess = np.repeat(T * X / N, N) ## VWAP

<ipython-input-5-1aec47da35a1>:11: RuntimeWarning: invalid value encountered in power
  * ((i - j + 1) ** (2 - gamma_const) - 2 * (i - j) ** (2 - gamma_const) +\
<ipython-input-5-1aec47da35a1>:12: RuntimeWarning: invalid value encountered in power
  (i - j - 1) ** (2 - gamma_const))


In [6]:
def gss_guess(t: np.array) -> np.array:
    gamma_1 = gamma((1 + gamma_const)/ 2)
    gamma_2 = gamma(1 + (gamma_const / 2))
    c = (X / N) / (np.sqrt(np.pi) * (T / 2) ** gamma_const * (gamma_1 / gamma_2))
    return (c / (t * (T - t)) ** ((1 - gamma_const) / 2))
gss_v_0_guess = gss_guess(np.arange(0, 1 + 1 / (N + 1), 1 / (N + 1)))[1:-1]

<ipython-input-6-b47e588dbda4>:5: RuntimeWarning: divide by zero encountered in true_divide
  return (c / (t * (T - t)) ** ((1 - gamma_const) / 2))


In [7]:
update_array = np.zeros((N, M))
update_array[:, 0] = v_guess
update_array_gss = np.zeros((N, M))
update_array_gss[:, 0] = gss_v_0_guess

In [8]:
def ham_series_solve(initial_array: np.array) -> np.array:
    update_dict = {p:0, d: delta_const}

    local_func = F_first(p)
    local_sub_func = F_second(p)
    for n in range(M - 1):
        for i in range(N):
            local_sum = 0
            for j in range(N):
                local_g = G[i, j]
                local_dict = update_dict.copy()
                for idx in range(M):
                    local_dict[v[idx]] = initial_array[j, idx]
                    local_dict[v_sub[idx]] = initial_array[i, idx]
                if j <= i:
                    local_f = local_func.subs(local_dict)
                else:
                    local_f = local_sub_func.subs(local_dict)
                local_sum += local_g * local_f
            initial_array[i, n + 1] = initial_array[i, n] + h * local_sum
        local_func = local_func.diff(p) / (n + 1)
        local_sub_func = local_sub_func.diff(p) / (n + 1)
    return initial_array

In [9]:
vwap_ham_solution = ham_series_solve(update_array)

In [10]:
gss_ham_solution = ham_series_solve(update_array_gss)

In [316]:
def impact_f(v):
    return np.sign(v) * np.abs(v) ** delta_const

In [317]:
def expect_cost_compute(v):
    return v @ A @ impact_f(v).T